In [1]:
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", None)

In [3]:
enrol_df = pd.read_csv("enrolment.csv")
demo_df = pd.read_csv("demographic.csv")
bio_df = pd.read_csv("biometric.csv")

In [4]:
all_dfs = [enrol_df, demo_df, bio_df]

In [5]:
enrol_df.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,01-09-2025,Goa,North Goa,403006,1,1,0
1,01-09-2025,Goa,North Goa,403114,1,0,0
2,01-09-2025,Goa,North Goa,403512,1,0,0
3,01-09-2025,Goa,North Goa,403529,1,0,0
4,01-09-2025,Goa,South Goa,403406,1,0,0


In [6]:
enrol_df["date"] = pd.to_datetime(enrol_df["date"], dayfirst=True)
demo_df["date"] = pd.to_datetime(demo_df["date"], dayfirst=True)
bio_df["date"] = pd.to_datetime(bio_df["date"], dayfirst=True)

In [7]:
print("Initial district counts:")
print(enrol_df["district"].nunique(), demo_df["district"].nunique(), bio_df["district"].nunique())

Initial district counts:
3 5 7


In [8]:
enrol_df["district"].value_counts().sort_index()

district
Bardez          2
North Goa    1092
South Goa     777
Name: count, dtype: int64

In [9]:
bio_df["district"].value_counts().sort_index()

district
Bardez               154
Bicholim               5
Dakshin Gangotri       1
North Goa           8894
Ponda                  1
South Goa           7078
Tiswadi                8
Name: count, dtype: int64

In [10]:
demo_df["district"].value_counts().sort_index()

district
Bardez         38
Bicholim        4
North Goa    3862
South Goa    3041
Tiswadi         2
Name: count, dtype: int64

In [11]:
import re

def clean_district_name(name):
    if pd.isna(name):
        return name
    name = str(name).lower().strip()
    name = name.replace('*', '') 
    name = re.sub(r'\s+', ' ', name) 
    name = name.strip()
    return name

print("=== Step 1: Basic Text Cleaning ===")
for df_name, df in [("Enrolment", enrol_df), ("Demographic", demo_df), ("Biometric", bio_df)]:
    before = df['district'].nunique()
    df['district'] = df['district'].apply(clean_district_name)
    after = df['district'].nunique()
    print(f"  {df_name}: {before} -> {after} unique districts")

=== Step 1: Basic Text Cleaning ===
  Enrolment: 3 -> 3 unique districts
  Demographic: 5 -> 5 unique districts
  Biometric: 7 -> 7 unique districts


In [12]:
cleanup_map = {
    "bardez": "north goa",
}

print("\n=== Step 2: Canonical Mapping ===")
for df_name, df in [("Enrolment", enrol_df), ("Demographic", demo_df), ("Biometric", bio_df)]:
    replacements = {}
    for old_name, new_name in cleanup_map.items():
        if old_name == new_name:
            continue
        count = (df['district'] == old_name).sum()
        if count > 0:
            replacements[old_name] = (new_name, count)
    
    df['district'] = df['district'].replace(cleanup_map)
    
    if replacements:
        print(f"\n{df_name}:")
        for old, (new, cnt) in sorted(replacements.items()):
            print(f"  '{old}' -> '{new}' ({cnt:,} rows)")
    else:
        print(f"\n{df_name}: No additional replacements needed")

print("\n=== Step 3: Final District Counts ===")
print(enrol_df["district"].nunique(), demo_df["district"].nunique(), bio_df["district"].nunique())


=== Step 2: Canonical Mapping ===

Enrolment:
  'bardez' -> 'north goa' (2 rows)

Demographic:
  'bardez' -> 'north goa' (38 rows)

Biometric:
  'bardez' -> 'north goa' (154 rows)

=== Step 3: Final District Counts ===
2 4 6


In [13]:
all_districts = sorted(set(enrol_df['district'].unique()) | 
                       set(demo_df['district'].unique()) | 
                       set(bio_df['district'].unique()))
print(f"Total unique districts across all datasets: {len(all_districts)}")
for d in all_districts:
    print(f"  - {d}")

Total unique districts across all datasets: 6
  - bicholim
  - dakshin gangotri
  - north goa
  - ponda
  - south goa
  - tiswadi


In [14]:
print(enrol_df["pincode"].nunique(), demo_df["pincode"].nunique(), bio_df["pincode"].nunique())

86 89 90


In [15]:
for df in all_dfs:
    df["month"] = df["date"].dt.month

In [16]:
enrol_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1871 entries, 0 to 1870
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            1871 non-null   datetime64[ns]
 1   state           1871 non-null   object        
 2   district        1871 non-null   object        
 3   pincode         1871 non-null   int64         
 4   age_0_5         1871 non-null   int64         
 5   age_5_17        1871 non-null   int64         
 6   age_18_greater  1871 non-null   int64         
 7   month           1871 non-null   int32         
dtypes: datetime64[ns](1), int32(1), int64(4), object(2)
memory usage: 109.8+ KB


In [17]:
demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6947 entries, 0 to 6946
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           6947 non-null   datetime64[ns]
 1   state          6947 non-null   object        
 2   district       6947 non-null   object        
 3   pincode        6947 non-null   int64         
 4   demo_age_5_17  6947 non-null   int64         
 5   demo_age_17_   6947 non-null   int64         
 6   month          6947 non-null   int32         
dtypes: datetime64[ns](1), int32(1), int64(3), object(2)
memory usage: 352.9+ KB


In [18]:
bio_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16141 entries, 0 to 16140
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          16141 non-null  datetime64[ns]
 1   state         16141 non-null  object        
 2   district      16141 non-null  object        
 3   pincode       16141 non-null  int64         
 4   bio_age_5_17  16141 non-null  int64         
 5   bio_age_17_   16141 non-null  int64         
 6   month         16141 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int64(3), object(2)
memory usage: 819.8+ KB


In [19]:
enrol_agg = enrol_df.groupby(["state", "district", "month"])[["age_0_5", "age_5_17", "age_18_greater"]].sum().reset_index()
demo_agg = demo_df.groupby(["state", "district", "month"])[["demo_age_5_17", "demo_age_17_"]].sum().reset_index()
bio_agg = bio_df.groupby(["state", "district", "month"])[["bio_age_5_17", "bio_age_17_"]].sum().reset_index()

combined_df = enrol_agg.merge(demo_agg, on = ["state", "district", "month"], how = "left").merge(bio_agg, on = ["state", "district", "month"], how = "left")
combined_df.fillna(0, inplace=True)
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_
0,Goa,north goa,1,42,2,18,20.0,303.0,469,279
1,Goa,north goa,6,154,64,20,0.0,0.0,1803,4303
2,Goa,north goa,7,21,10,11,0.0,0.0,1357,3649
3,Goa,north goa,9,411,64,40,202.0,3110.0,1945,2901
4,Goa,north goa,10,226,25,6,206.0,2188.0,3607,3039


In [20]:
combined_df["E"] = combined_df["age_0_5"] + combined_df["age_5_17"] + combined_df["age_18_greater"]
combined_df["DU"] = combined_df["demo_age_5_17"] + combined_df["demo_age_17_"]
combined_df["BU"] = combined_df["bio_age_5_17"] + combined_df["bio_age_17_"]
combined_df["U"] = combined_df["DU"] + combined_df["BU"]
combined_df["T"] = combined_df["E"] + combined_df["U"]
combined_df.head(20)

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T
0,Goa,north goa,1,42,2,18,20.0,303.0,469,279,62,323.0,748,1071.0,1133.0
1,Goa,north goa,6,154,64,20,0.0,0.0,1803,4303,238,0.0,6106,6106.0,6344.0
2,Goa,north goa,7,21,10,11,0.0,0.0,1357,3649,42,0.0,5006,5006.0,5048.0
3,Goa,north goa,9,411,64,40,202.0,3110.0,1945,2901,515,3312.0,4846,8158.0,8673.0
4,Goa,north goa,10,226,25,6,206.0,2188.0,3607,3039,257,2394.0,6646,9040.0,9297.0
5,Goa,north goa,11,299,38,21,373.0,5143.0,4071,3410,358,5516.0,7481,12997.0,13355.0
6,Goa,north goa,12,160,13,16,314.0,4529.0,3019,2662,189,4843.0,5681,10524.0,10713.0
7,Goa,south goa,1,35,8,5,20.0,327.0,277,188,48,347.0,465,812.0,860.0
8,Goa,south goa,6,155,31,8,0.0,0.0,1878,3843,194,0.0,5721,5721.0,5915.0
9,Goa,south goa,9,282,41,34,201.0,2669.0,1737,2127,357,2870.0,3864,6734.0,7091.0


In [21]:
district_monthly_counts = combined_df.groupby(["district", "month"]).agg(total_months = ("month", "count"), active_months = ("T", lambda x : (x > 0).sum())).reset_index()
district_monthly_counts.head()

,district,month,total_months,active_months
0,north goa,1,1,1
1,north goa,6,1,1
2,north goa,7,1,1
3,north goa,9,1,1
4,north goa,10,1,1


In [22]:
district_monthly_counts["zero_months"] = district_monthly_counts["total_months"] - district_monthly_counts["active_months"]
district_monthly_counts["activity_ratio"] = district_monthly_counts["active_months"] / district_monthly_counts["total_months"]
district_monthly_counts["zero_month_ratio"] = district_monthly_counts["zero_months"] / district_monthly_counts["total_months"]

combined_df = combined_df.merge(district_monthly_counts[["district", "month", "activity_ratio", "zero_month_ratio"]], on = ["district", "month"], how = "left")
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio
0,Goa,north goa,1,42,2,18,20.0,303.0,469,279,62,323.0,748,1071.0,1133.0,1.0,0.0
1,Goa,north goa,6,154,64,20,0.0,0.0,1803,4303,238,0.0,6106,6106.0,6344.0,1.0,0.0
2,Goa,north goa,7,21,10,11,0.0,0.0,1357,3649,42,0.0,5006,5006.0,5048.0,1.0,0.0
3,Goa,north goa,9,411,64,40,202.0,3110.0,1945,2901,515,3312.0,4846,8158.0,8673.0,1.0,0.0
4,Goa,north goa,10,226,25,6,206.0,2188.0,3607,3039,257,2394.0,6646,9040.0,9297.0,1.0,0.0


In [23]:
print("Columns:", combined_df.columns.tolist())

Columns: ['state', 'district', 'month', 'age_0_5', 'age_5_17', 'age_18_greater', 'demo_age_5_17', 'demo_age_17_', 'bio_age_5_17', 'bio_age_17_', 'E', 'DU', 'BU', 'U', 'T', 'activity_ratio', 'zero_month_ratio']


In [24]:
district_volume_metrics = combined_df.groupby(["state", "district"]).agg(avg_monthly_enrolment = ("E", "mean"),
                                               monthly_valatility = ("T", lambda x: x.std(ddof=0) / x.mean() if x.mean() > 0 else 0),
                                               peak_load_ratio = ("T", lambda x: x.max() / x.mean() if x.mean() > 0 else 0)).reset_index()


combined_df = combined_df.merge(district_volume_metrics, on=["state", "district"], how="left")
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio,avg_monthly_enrolment,monthly_valatility,peak_load_ratio
0,Goa,north goa,1,42,2,18,20.0,303.0,469,279,62,323.0,748,1071.0,1133.0,1.0,0.0,237.285714,0.47631,1.713341
1,Goa,north goa,6,154,64,20,0.0,0.0,1803,4303,238,0.0,6106,6106.0,6344.0,1.0,0.0,237.285714,0.47631,1.713341
2,Goa,north goa,7,21,10,11,0.0,0.0,1357,3649,42,0.0,5006,5006.0,5048.0,1.0,0.0,237.285714,0.47631,1.713341
3,Goa,north goa,9,411,64,40,202.0,3110.0,1945,2901,515,3312.0,4846,8158.0,8673.0,1.0,0.0,237.285714,0.47631,1.713341
4,Goa,north goa,10,226,25,6,206.0,2188.0,3607,3039,257,2394.0,6646,9040.0,9297.0,1.0,0.0,237.285714,0.47631,1.713341


In [25]:
district_update_burden = combined_df.groupby(["state", "district"]).agg(avg_monthly_enrollments = ("E", "sum"), avg_monthly_demo_updates = ("DU", "sum"), avg_monthly_bio_updates = ("BU", "sum")).reset_index()

district_update_burden["U"] = district_update_burden["avg_monthly_demo_updates"] + district_update_burden["avg_monthly_bio_updates"]
district_update_burden["biometric_burden"] = district_update_burden["avg_monthly_bio_updates"] / (district_update_burden["avg_monthly_bio_updates"] + district_update_burden["avg_monthly_demo_updates"])
district_update_burden["update_dominant"] = np.where(district_update_burden["U"] > district_update_burden["avg_monthly_enrollments"], 1, 0)
district_update_burden["enrollment_update_balance"] = district_update_burden["avg_monthly_enrollments"] / (district_update_burden["avg_monthly_enrollments"] + district_update_burden["U"])

combined_df = combined_df.merge(district_update_burden[["state", "district", "biometric_burden", "update_dominant", "enrollment_update_balance"]], on=["state", "district"], how="left")
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio,avg_monthly_enrolment,monthly_valatility,peak_load_ratio,biometric_burden,update_dominant,enrollment_update_balance
0,Goa,north goa,1,42,2,18,20.0,303.0,469,279,62,323.0,748,1071.0,1133.0,1.0,0.0,237.285714,0.47631,1.713341,0.69022,1,0.030442
1,Goa,north goa,6,154,64,20,0.0,0.0,1803,4303,238,0.0,6106,6106.0,6344.0,1.0,0.0,237.285714,0.47631,1.713341,0.69022,1,0.030442
2,Goa,north goa,7,21,10,11,0.0,0.0,1357,3649,42,0.0,5006,5006.0,5048.0,1.0,0.0,237.285714,0.47631,1.713341,0.69022,1,0.030442
3,Goa,north goa,9,411,64,40,202.0,3110.0,1945,2901,515,3312.0,4846,8158.0,8673.0,1.0,0.0,237.285714,0.47631,1.713341,0.69022,1,0.030442
4,Goa,north goa,10,226,25,6,206.0,2188.0,3607,3039,257,2394.0,6646,9040.0,9297.0,1.0,0.0,237.285714,0.47631,1.713341,0.69022,1,0.030442


In [26]:
cols_to_drop = [c for c in combined_df.columns if c.endswith('_y')]
if cols_to_drop:
    combined_df.drop(cols_to_drop, axis=1, inplace=True)
    combined_df.rename(columns={c: c.replace('_x', '') for c in combined_df.columns if c.endswith('_x')}, inplace=True)
print("Columns after cleanup:", combined_df.columns.tolist())

Columns after cleanup: ['state', 'district', 'month', 'age_0_5', 'age_5_17', 'age_18_greater', 'demo_age_5_17', 'demo_age_17_', 'bio_age_5_17', 'bio_age_17_', 'E', 'DU', 'BU', 'U', 'T', 'activity_ratio', 'zero_month_ratio', 'avg_monthly_enrolment', 'monthly_valatility', 'peak_load_ratio', 'biometric_burden', 'update_dominant', 'enrollment_update_balance']


In [27]:
combined_df = combined_df.groupby(["state", "district"], as_index = False).first()
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio,avg_monthly_enrolment,monthly_valatility,peak_load_ratio,biometric_burden,update_dominant,enrollment_update_balance
0,Goa,north goa,1,42,2,18,20.0,303.0,469,279,62,323.0,748,1071.0,1133.0,1.0,0.0,237.285714,0.476310,1.713341,0.690220,1,0.030442
1,Goa,south goa,1,35,8,5,20.0,327.0,277,188,48,347.0,465,812.0,860.0,1.0,0.0,204.333333,0.438207,1.538419,0.621519,1,0.031090


In [28]:
def normalize(x):
    maxx, minx = x.max(), x.min()
    if maxx == minx:
        return x * 0 + 0.5
    normalized = (x - minx) / (maxx - minx)
    return normalized

def inverse_normalize(x):
    inversed = 1 - normalize(x)
    return inversed

In [29]:
combined_df["access"] = (combined_df["activity_ratio"] + normalize(combined_df["avg_monthly_enrolment"])) / 2
combined_df["responsiveness"] = normalize(combined_df["U"] / (combined_df["E"] + combined_df["U"]))
combined_df["inclusion"] = normalize((combined_df["age_0_5"] + combined_df["age_5_17"]) / combined_df["E"])
combined_df["stability"] = (inverse_normalize(combined_df["monthly_valatility"]) + inverse_normalize(combined_df["peak_load_ratio"])) / 2
combined_df["visibility"] = combined_df["activity_ratio"]

combined_df["DEI"] = (combined_df["access"] + combined_df["responsiveness"] + combined_df["inclusion"] + combined_df["stability"] + combined_df["visibility"]) / 5
combined_df["ASS"] = (inverse_normalize(combined_df["activity_ratio"]) + inverse_normalize(combined_df["avg_monthly_enrolment"])) / 2
combined_df["UBS"] = (normalize(combined_df["biometric_burden"]) + normalize(combined_df["update_dominant"])) / 2
combined_df["SRS"] = (normalize(combined_df["monthly_valatility"]) + normalize(combined_df["zero_month_ratio"])) / 2

combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio,avg_monthly_enrolment,monthly_valatility,peak_load_ratio,biometric_burden,update_dominant,enrollment_update_balance,access,responsiveness,inclusion,stability,visibility,DEI,ASS,UBS,SRS
0,Goa,north goa,1,42,2,18,20.0,303.0,469,279,62,323.0,748,1071.0,1133.0,1.0,0.0,237.285714,0.476310,1.713341,0.690220,1,0.030442,1.0,1.0,0.0,0.0,1.0,0.6,0.25,0.75,0.75
1,Goa,south goa,1,35,8,5,20.0,327.0,277,188,48,347.0,465,812.0,860.0,1.0,0.0,204.333333,0.438207,1.538419,0.621519,1,0.031090,0.5,0.0,1.0,1.0,1.0,0.7,0.75,0.25,0.25


In [30]:
combined_df.drop(["access", "responsiveness", "inclusion", "stability", "visibility"], axis=1, inplace=True)
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio,avg_monthly_enrolment,monthly_valatility,peak_load_ratio,biometric_burden,update_dominant,enrollment_update_balance,DEI,ASS,UBS,SRS
0,Goa,north goa,1,42,2,18,20.0,303.0,469,279,62,323.0,748,1071.0,1133.0,1.0,0.0,237.285714,0.476310,1.713341,0.690220,1,0.030442,0.6,0.25,0.75,0.75
1,Goa,south goa,1,35,8,5,20.0,327.0,277,188,48,347.0,465,812.0,860.0,1.0,0.0,204.333333,0.438207,1.538419,0.621519,1,0.031090,0.7,0.75,0.25,0.25


In [31]:
print(f"Total districts: {len(combined_df)}")
print(f"\nScore summary statistics:")
for col in ['DEI', 'ASS', 'UBS', 'SRS']:
    print(f"  {col}: min={combined_df[col].min():.4f}, max={combined_df[col].max():.4f}, mean={combined_df[col].mean():.4f}")

for col in ['DEI', 'ASS', 'UBS', 'SRS']:
    assert combined_df[col].min() >= 0, f"{col} has negative values"
    assert combined_df[col].max() <= 1, f"{col} exceeds 1"
print("\n✓ All scores in valid [0, 1] range")

Total districts: 2

Score summary statistics:
  DEI: min=0.6000, max=0.7000, mean=0.6500
  ASS: min=0.2500, max=0.7500, mean=0.5000
  UBS: min=0.2500, max=0.7500, mean=0.5000
  SRS: min=0.2500, max=0.7500, mean=0.5000

✓ All scores in valid [0, 1] range


In [32]:
combined_df.to_csv("goa_district_analysis.csv", index=False)
final_df = combined_df[["state", "district", "DEI", "ASS", "UBS", "SRS"]]
final_df.to_csv("goa_district_final_scores.csv", index=False)
print("✓ Saved goa_district_analysis.csv")
print("✓ Saved goa_district_final_scores.csv")

✓ Saved goa_district_analysis.csv
✓ Saved goa_district_final_scores.csv


In [33]:
final_df.sort_values('DEI', ascending=False)

,state,district,DEI,ASS,UBS,SRS
1,Goa,south goa,0.7,0.75,0.25,0.25
0,Goa,north goa,0.6,0.25,0.75,0.75
